In [76]:
from selenium import webdriver
import pandas as pd

In [82]:
url = "https://www.eloratings.net/2020_results"
driver = webdriver.Chrome("chromedriver.exe")
driver.get(url)

In [83]:
row_list = driver.find_elements_by_xpath('//*[@id="maintable_2020_results"]/div[5]/div/div')

In [84]:
columns = ['date', 'tournament', 'home_team', 'away_team', 'home_team_rating', 'away_team_rating', 'home_goals', 'away_goals', 
           'home_team_rating_change', 'away_team_rating_change', 'home_team_new_rating', 'away_team_new_rating', 
           'home_team_rank', 'away_team_rank', 'home_team_rank_change', 'away_team_rank_change', 
           'home_team_new_rank', 'away_team_new_rank']

In [85]:
df_row_list = []
for i, row in enumerate(row_list):
    date = row.find_element_by_xpath('.//div[1]').text.replace("\n", " ")
    tournament = row.find_element_by_xpath('.//div[4]').text.replace("\n", " ")
    teams = row.find_element_by_xpath('.//div[2]').text
    home_team = teams.split("\n")[0]
    away_team = teams.split("\n")[1]
    rating_changes = row.find_element_by_xpath('.//div[5]').text.replace("−", "-")
    home_team_rating_change = int(rating_changes.split("\n")[0])
    away_team_rating_change = int(rating_changes.split("\n")[1])
    ratings = row.find_element_by_xpath('.//div[6]').text
    home_team_new_rating = int(ratings.split("\n")[0])
    away_team_new_rating = int(ratings.split("\n")[1])
    home_team_rating = home_team_new_rating - home_team_rating_change
    away_team_rating = away_team_new_rating - away_team_rating_change
    rank_changes = row.find_element_by_xpath('.//div[7]').text.replace("−", "-")
    home_team_rank_change = int(rank_changes.split("\n")[0])
    away_team_rank_change = int(rank_changes.split("\n")[1])
    ranks = row.find_element_by_xpath('.//div[8]').text
    home_team_new_rank = int(ranks.split("\n")[0])
    away_team_new_rank = int(ranks.split("\n")[1])
    home_team_rank = home_team_new_rank - home_team_rank_change
    away_team_rank = away_team_new_rank - away_team_rank_change
    result = row.find_element_by_xpath('.//div[3]').text
    home_goals = int(result.split("\n")[0])
    away_goals = int(result.split("\n")[1])
    df_row = [date, tournament, home_team, away_team, home_team_rating, away_team_rating, home_goals, away_goals, 
              home_team_rating_change, away_team_rating_change, home_team_new_rating, away_team_new_rating, 
              home_team_rank, away_team_rank, home_team_rank_change, away_team_rank_change, 
              home_team_new_rank, away_team_new_rank]
    df_row_list.append(df_row)
    print(i)
driver.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [88]:
df = pd.DataFrame(df_row_list, columns=columns)
df

,date,tournament,home_team,away_team,home_team_rating,away_team_rating,home_team_rating_change,away_team_rating_change,home_team_new_rating,away_team_new_rating,home_team_rank,away_team_rank,home_team_rank_change,away_team_rank_change,home_team_new_rank,away_team_new_rank,home_goals,away_goals
0,Jan 7 2020,Friendly in the United States,Canada,Barbados,1604,1105,2,-2,1606,1103,58,182,0,-1,58,181,4,1
1,Jan 9 2020,Friendly in Qatar,Sweden,Moldova,1860,1240,1,-1,1861,1239,18,147,0,0,18,147,1,0
2,Jan 10 2020,Friendly in the United States,Canada,Barbados,1606,1103,2,-2,1608,1101,56,181,1,0,57,181,4,1
3,Jan 12 2020,Friendly in Qatar,Sweden,Kosovo,1861,1564,3,-3,1864,1561,18,66,0,0,18,66,1,0
4,Jan 15 2020,Friendly in the United States,Iceland,Canada,1672,1608,8,-8,1680,1600,43,59,1,-1,44,58,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,Nov 18 2020,European Nations League A in England,England,Iceland,1956,1639,6,-6,1962,1633,9,49,0,0,9,49,4,0
362,Nov 18 2020,European Nations League A in Bosnia and Herzeg...,Bosnia and Herzegovina,Italy,1646,1986,-12,12,1634,1998,49,5,-1,1,48,6,0,2
363,Nov 18 2020,European Nations League A in Belgium,Belgium,Denmark,2100,1938,11,-11,2111,1927,2,14,0,-1,2,13,4,2
364,Dec 4 2020,World Cup & Asian Cup qual in Qatar,Qatar,Bangladesh,1640,842,0,0,1640,842,47,207,0,0,47,207,5,0
